In [ ]:
using Pkg;
#Pkg.add("Wavelets");Pkg.add("ImageFiltering")
using Wavelets, TestImages,Plots,ImageFiltering

# Déconvolution dans une base d'ondelettes

On veut résoudre le problème de déconvolution
$$y=Ax+b \quad \mbox{où} \quad(Ax)(s)=\int h(t)x(s-t) dt  $$
et $h$ est la réponse impulsionnelle et $b$ est un bruit.

Néanmoins, on va essayer de reconstruire des signaux différents de signaux parcimonieux dans la base canonique.


In [ ]:
# Création du signal original
n=128 # taille du signal (prendre une puissance de 2)
x0 = testfunction(n, "Blocks") # on peut essayer aussi Bumps, HeaviSine
plot(x0,label="Signal original")

1. Observer les coefficients d'ondelettes du signal. Commenter.

In [ ]:
# Observation des coefficients en ondelette
wt= wavelet(WT.haar,WT.SymBound)
#wt = wavelet(WT.db3, WT.SymBound)
c=dwt(x0,wt) # Coefficients de x0 dans la base d'ondelettes
plot(c,lines=:stem,marker=:circle)

In [ ]:
# Filtrage: noyau gaussien
largeur=1.0 # largeur de la gaussienne
h = ImageFiltering.Kernel.gaussian((largeur,))

xfilt = imfilter(x0, h)
sigma=0.0
bruit=sigma*randn(size(xfilt))
y=xfilt+bruit
plot(xfilt,label="Filtré")
plot!(y,label="Observation")

## Résolution par la transformée en ondelettes

On va se placer dans la base des coefficients d'ondelettes et résoudre

$$ \min \lambda\|c\|_1 +1/2 \| AWc-y\|^2$$
où $A$ est l'opérateur de convolution et $W$ l'application qui à des coefficients d'ondelettes reconstruit la fonction associée.

In [ ]:
# Quelques fonctions auxiliaires

function soft_thresh(c,seuil)
    return (sign(c)*max(abs(c)-seuil,0.)) 
end    


function applyAW(c)
    global wt
    global h
    x=idwt(c, wt)#signal
    return imfilter(x, reflect(h),"reflect") 
    #return imfilter(x, reflect(h),"circular") 
end
function applyAWstar(x)
    global wt
    global h
    xfilt=imfilter(x, h,"reflect")
    #xfilt=imfilter(x, h,"circular")
    return dwt(xfilt,wt)# coefficients d'ondelette
end

"""
    fwbackward(applyA!,applyAstar!,λ,τ,niter,xinit=zeros(size(A,2)))
Applique l'algorithme ISTA avec l'opérateur A donné à travers les fonctions applyA et applyAstar.
On cherche à résoudre
min λ||x||_1 +1/2 |Ax-y|^2

tau : pas de la descente de gradient (forward)
niter: nombre d'itérations
"""
function fwbackward(applyAW,applyAWstar,y,λ,τ,niter,cinit)
    c=copy(cinit)
    
    for k=1:niter
        c.= c.-τ*applyAWstar(applyAW(c)-y)
        c.=soft_thresh.(c,λ*τ)
    end
    return c
end

2. Sachant que la norme d'un opérateur de convolution est $\|h\|_1$ et que la matrice de la transformée en ondelettes est orthogonale, donner (un majorant de) la norme de $AW$.

In [ ]:
# normA=  # A COMPLETER
#tau= # A COMPLETER
λ=0.01
crec=fwbackward(applyAW,applyAWstar,y,λ,tau,10000,zeros(n))
xrec=idwt(crec,wt);
plot(x0,label="Signal recherché")
plot!(y,label="Observation")
plot!(xrec,label="Signal recontruit")

## Reconstruction d'images

3. Charger une image et observer ses coefficients d'ondelettes. Commenter.

In [ ]:
using Images, Plots, TestImages
img=Float32.(Gray.(testimage("house"))) # cameraman, house,coffee, chelsea... voir la liste sur https://testimages.juliaimages.org/
M,N=size(img)
plot(Gray.(img),showaxis=false,title="Image originale")

wt = wavelet(WT.haar, WT.SymBound)
#On peut aussi essayer un autre type d'ondelette. Dans ce cas, il faut réévaluer la cellule de applyAW et applyAWstar
#wt = wavelet(WT.db3, WT.SymBound)


L = 3 # Nombre de niveaux d'ondelette
xts = wplotim(img, L, wt)

display(plot(Gray.(img),showaxis=false,title="Image originale"))
plot(Gray.(xts),showaxis=false,title="Coefficients ondelette")

4. Essayer différentes reconstruction pour différentes valeurs de $\sigma$ et $\lambda$. On pourra aussi changer de type d'ondelette.

In [ ]:
# Filtrage: noyau gaussien
largeur=3.0 # largeur de la gaussienne
h = ImageFiltering.Kernel.gaussian(largeur)

xfilt = imfilter(img, h,"reflect")
sigma=0.0
bruit=sigma*randn(size(xfilt))
y=xfilt+bruit
#plot(xfilt,label="Filtré")
plot(Gray.(y),title="Observation")

In [ ]:
# normA=  # A COMPLETER
#tau= # A COMPLETER
λ=0.001
crec=fwbackward(applyAW,applyAWstar,y,λ,tau,400,zeros(M,N)) # reconstruction des coefficients
xrec=idwt(crec,wt); # reconstruction de l'image
display(plot(Gray.(y),title="Image observée"))
display(plot(Gray.(xrec),title="Image reconstruite"))